In [1]:
import os

curr_dir = os.path.split(os.getcwd())[1]
if curr_dir != "irl-environment-design":
    os.chdir("..")

import numpy as np

from src.utils.make_environment import (
    transition_matrix,
    Environment,
    insert_walls_into_T,
)

from src.utils.constants import GenParamTuple# candidate_environments_args["n_environments"] = 50


np.set_printoptions(linewidth=160, precision=2)

from src.utils.environment_design import EnvironmentDesign

## Make true environment

In [2]:
# 0. Setup
## 0.2 Setup the environment
N, M = 7,7

# TODO describe world
agent_R = np.zeros((N, M))
agent_R[N-1, 0] = 1
agent_R[N-1, M-1] = 3



# Start by making the agent we want to learn the policy of
agent_p = 0.7
agent_gamma = 0.6
p_true=0.7

agent_R = agent_R.flatten()
goal_states = np.where(agent_R != 0)[0]


wall_states = [14] #TODO: why do we need this

T_true = transition_matrix(N, M, p=p_true, absorbing_states=goal_states)
T_True = insert_walls_into_T(T=T_true, wall_indices=wall_states)


def custom_transition_func(p):

    _T = transition_matrix(N=7, M=7, p=p, absorbing_states=goal_states)
    _T = insert_walls_into_T(T=_T, wall_indices=wall_states)
    return _T

def custom_gamma_func(gamma):
    return gamma

def custom_reward_func():
    return agent_R

#Create parameter ranges
resolution = 15
p_range = np.linspace(0.5, 0.9, resolution)
gamma_range = np.linspace(0.1, 0.9, resolution)

p_range_1 = np.linspace(start = 1, stop = 5, num = resolution)
p_range_2 = np.linspace(start = 1, stop = 5, num = resolution)
p_range = np.array([p_range_1, p_range_2])

gamma_range = gamma_range.reshape(1, resolution)

true_params = GenParamTuple(T = custom_transition_func(agent_p), gamma=agent_gamma, R=agent_R)




big_small = Environment(
    N=N,
    M=M,
    reward_function = custom_reward_func,
    transition_function=custom_transition_func,
    gamma = custom_gamma_func,
    wall_states=wall_states,
    start_state=0,
    goal_states=goal_states
)

In [3]:
env_design = EnvironmentDesign(base_environment=big_small, 
                               user_params=true_params, 
                               learn_what = ["gamma", "T"],
                               parameter_ranges_R=None,
                               parameter_ranges_gamma=gamma_range,
                               parameter_ranges_T=p_range,)

Generated parameter mesh of shape:  (15, 15, 15)


In [8]:
candidate_environments_args = {}
candidate_environments_args["generate_how"] = "entropy_BM"

env_design.run_n_episodes(n_episodes = 5,
                          candidate_environments_args=candidate_environments_args)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3375, 3) + inhomogeneous part.

In [ ]:
'''
user parameters consist of
- parametrizations of reward, transition and gamma that user supplies
- ranges for the unknown parameters
- resolution of the grid search
'''

'\nuser parameters consist of\n- parametrizations of reward, transition and gamma that user supplies\n- ranges for the unknown parameters\n- resolution of the grid search\n'

## Make custom parametrizations

In [ ]:
def custom_big_small_reward_func(N, M, reward_ratio: float):

    '''
    Generates a big-small like reward function.

    Args:
    - N: int, number of rows in the grid
    - M: int, number of columns in the grid
    - reward_ratio: float, ratio of the reward in the bottom right corner to the reward at the bottom left corner
    '''

    _R = np.zeros((N, M))
    _R[N-1, 0] = 1
    _R[N-1, M-1] = reward_ratio

def custom_T(N, M, p: float, goal_states, wall_states):

    _T = transition_matrix(N, M, p=p, absorbing_states=goal_states)
    _T = insert_walls_into_T(T=_T, wall_indices=wall_states)

    return _T

resolution = 15
range_rewards_1 = np.linspace(start = 1, stop = 5, num = resolution)
range_rewards_2 = np.linspace(start = 1, stop = 5, num = resolution)
range_rewards = np.array([range_rewards_1, range_rewards_2])

range_p = np.linspace(start = 0.5, stop = 0.95, num = resolution)
range_gamma = np.linspace(start = 0.5, stop = 0.95, num = resolution)


In [ ]:
range_rewards.shape

(2, 15)

In [ ]:
np.empty((range_rewards.shape, range_rewards.shape)).shape

TypeError: 'tuple' object cannot be interpreted as an integer

In [ ]:
parameter_ranges = []
for rew in range_rewards:
    parameter_ranges.append(rew)
for p in range_p:
    parameter_ranges.append(p)

In [ ]:
parameter_ranges

[array([1.        , 1.28571429, 1.57142857, 1.85714286, 2.14285714,
        2.42857143, 2.71428571, 3.        , 3.28571429, 3.57142857,
        3.85714286, 4.14285714, 4.42857143, 4.71428571, 5.        ]),
 array([1.        , 1.28571429, 1.57142857, 1.85714286, 2.14285714,
        2.42857143, 2.71428571, 3.        , 3.28571429, 3.57142857,
        3.85714286, 4.14285714, 4.42857143, 4.71428571, 5.        ])]

In [ ]:
import itertools

for idx, param in enumerate(itertools.product(*parameter_ranges)):
    print(param)
    print(idx)

(1.0, 1.0)
0
(1.0, 1.2857142857142856)
1
(1.0, 1.5714285714285714)
2
(1.0, 1.8571428571428572)
3
(1.0, 2.142857142857143)
4
(1.0, 2.4285714285714284)
5
(1.0, 2.7142857142857144)
6
(1.0, 3.0)
7
(1.0, 3.2857142857142856)
8
(1.0, 3.571428571428571)
9
(1.0, 3.8571428571428568)
10
(1.0, 4.142857142857142)
11
(1.0, 4.428571428571429)
12
(1.0, 4.7142857142857135)
13
(1.0, 5.0)
14
(1.2857142857142856, 1.0)
15
(1.2857142857142856, 1.2857142857142856)
16
(1.2857142857142856, 1.5714285714285714)
17
(1.2857142857142856, 1.8571428571428572)
18
(1.2857142857142856, 2.142857142857143)
19
(1.2857142857142856, 2.4285714285714284)
20
(1.2857142857142856, 2.7142857142857144)
21
(1.2857142857142856, 3.0)
22
(1.2857142857142856, 3.2857142857142856)
23
(1.2857142857142856, 3.571428571428571)
24
(1.2857142857142856, 3.8571428571428568)
25
(1.2857142857142856, 4.142857142857142)
26
(1.2857142857142856, 4.428571428571429)
27
(1.2857142857142856, 4.7142857142857135)
28
(1.2857142857142856, 5.0)
29
(1.5714285714

In [ ]:
np.zeros(3*[10]).shape

(10, 10, 10)

In [ ]:
parameter_ranges = np.array([[0.5, 0.95], [0.5, 0.95]])

In [ ]:
env_design = EnvironmentDesign(base_environment=big_small, user_params=true_params, learn_what = "gamma, T")

In [ ]:
ranges = np.array([])


In [ ]:
import numpy as np

a = np.array([[2, 2]])
ranges = np.concatenate((ranges, a))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 1 dimension(s) and the array at index 1 has 2 dimension(s)

In [ ]:
gamma_ranges = np.array([[0.5, 0.95], [1,1]])
gamma_ranges

array([[0.5 , 0.95],
       [1.  , 1.  ]])

In [ ]:
gamma_ranges.reshape(1,4)

array([[0.5 , 0.95, 1.  , 1.  ]])